# [MLP_base] 기본 딥러닝 모델

### 👨‍💻 Developer's Log
- **Situation**: 드디어 Churn Prediction 프로젝트가 시작되었다. 팀원들은 SOTA 모델(Transformer, TabNet)을 쓰자고 난리지만, 나는 반대다.
- **Thought**: 데이터가 깨끗한지, 불균형은 해결되는지 확인하는 게 먼저다. 복잡한 모델은 나중에 생각하자.
- **Action**: 가장 단순한 **3-Layer MLP**로 Baseline을 잡는다. 여기서 점수가 안 나오면 모델 문제가 아니라 데이터 문제다.


In [19]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt

# [한글 폰트 설정]
plt.rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False

### 👨‍💻 Developer's Log: 데이터 준비
- **고민**: `user_id`가 제각각이라 병합이 안 된다.
- **해결**: 전부 String으로 강제 형변환을 해서 Key를 맞췄다. 이런 사소한 게 나중에 큰 버그를 만든다.
- **전략**: `Inner Join`으로 데이터가 완벽한 유저만 남긴다. 결측치 채우느라 머리 쓰기보다, 확실한 데이터로 승부하겠다.

In [20]:
print('>>> [MLP_base] Loading Data...')
base_path = '/Users/gimdabin/Deep-Learning/project_root/outputs/samples/'
anchors = pd.read_parquet(base_path + 'anchors.parquet')
features = pd.read_parquet(base_path + 'features_ml_clean.parquet')
labels = pd.read_parquet(base_path + 'labels.parquet')

# [ID 타입 통일]
anchors['user_id'] = anchors['user_id'].astype(str)
features['user_id'] = features['user_id'].astype(str)
labels['user_id'] = labels['user_id'].astype(str)

# [데이터 병합]
data = anchors.merge(features, on=['user_id', 'anchor_time'], how='inner')
data = data.merge(labels, on=['user_id', 'anchor_time'], how='inner')
data['target'] = (data['label'] == 'm2').astype(int)

feature_cols = [c for c in features.columns if c not in ['user_id', 'anchor_time']]
X = data[feature_cols].copy().fillna(0)
y = data['target'].values

>>> [MLP_base] Loading Data...


### 👨‍💻 Developer's Log: 불균형과의 전쟁
- **발견**: 이탈 유저(Class 1)가 너무 적다. 이대로면 모델이 "전부 비이탈자입니다"라고 찍어도 정확도 90%가 나올 판이다.
- **결단**: **SMOTE**를 쓰자. 데이터를 뻥튀기해서 5:5로 밸런스를 맞춘다. 
- **원칙**: 단, `Test Set`은 절대 건드리지 않는다. 연습문제(Train)는 쉽게 내더라도, 수능(Test)은 현실과 똑같이 봐야 하니까.

In [21]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

print(f'Before SMOTE: {np.bincount(y_train)} (0: Non-Churn, 1: Churn)')
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)
print(f'After SMOTE:  {np.bincount(y_train_res)} (Balanced!)')

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_res)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

train_loader = DataLoader(TensorDataset(torch.FloatTensor(X_train_scaled), torch.FloatTensor(y_train_res)), batch_size=256, shuffle=True)
val_loader = DataLoader(TensorDataset(torch.FloatTensor(X_val_scaled), torch.FloatTensor(y_val)), batch_size=256)
test_loader = DataLoader(TensorDataset(torch.FloatTensor(X_test_scaled), torch.FloatTensor(y_test)), batch_size=256)

Before SMOTE: [ 88663 399461] (0: Non-Churn, 1: Churn)
After SMOTE:  [399461 399461] (Balanced!)


### 👨‍💻 Developer's Log: 모델 설계
- **철학**: "Simple is Best". 층을 깊게 쌓을 필요도 없다.
- **구조**: Input -> Hidden(64) -> Hidden(32) -> Output(1). 끝.
- **기대**: 이 단순한 구조로도 의미 있는 점수가 나온다면, 우리 데이터에는 '패턴'이 있다는 증거다.

In [22]:
class BasicMLP(nn.Module):
    def __init__(self, input_dim, hidden_dim=64):
        super(BasicMLP, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Linear(hidden_dim // 2, 1)
        )
    def forward(self, x):
        return self.net(x)

model = BasicMLP(X.shape[1])
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [23]:
print('>>> Training Start...')
epochs = 10
for epoch in range(epochs):
    model.train()
    epoch_loss = 0
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs).squeeze()
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    print(f'[Epoch {epoch+1}/{epochs}] Loss: {epoch_loss/len(train_loader):.4f}')

>>> Training Start...
[Epoch 1/10] Loss: 0.6256
[Epoch 2/10] Loss: 0.6175
[Epoch 3/10] Loss: 0.6140
[Epoch 4/10] Loss: 0.6119
[Epoch 5/10] Loss: 0.6103
[Epoch 6/10] Loss: 0.6088
[Epoch 7/10] Loss: 0.6075
[Epoch 8/10] Loss: 0.6063
[Epoch 9/10] Loss: 0.6052
[Epoch 10/10] Loss: 0.6039


### 👨‍💻 Developer's Log: 결과 분석 및 다음 스텝
- **성과**: Top 10% 지점에서 괜찮은 Lift가 나온다. 데이터 품질은 합격이다.
- **문제점**: 학습 Loss가 출렁거린다(Oscillation). 모델이 단순해서 과적합(Overfitting)되거나 학습이 불안정한 것 같다.
- **Next Step**: "불안정함을 잡아야 한다." 다음 모델(MLP_enhance)에서는 **Batch Normalization**과 **Dropout**을 도입해서 학습을 안정화시켜보자. 

👉 **[Next: MLP_enhance.ipynb - The Stabilizer]**

In [24]:
# --- 5. Advanced Verification (Google DeepMind Style) ---
from sklearn.metrics import roc_auc_score, average_precision_score, brier_score_loss, roc_curve, precision_recall_curve, f1_score
from sklearn.calibration import calibration_curve
from scipy.stats import ks_2samp

print('>>> Generating Advanced Verification Dashboard...')

# [평가 모드] 모델 평가 모드 전환
model.eval()
all_logits = []
all_targets = []

# [추론] 정답 없이 예측
with torch.no_grad():
    for inputs, targets in test_loader:
        outputs = model(inputs)
        all_logits.extend(outputs.squeeze().numpy())
        all_targets.extend(targets.numpy())

# [확률 변환]
all_probs = 1 / (1 + np.exp(-np.array(all_logits)))
df_res = pd.DataFrame({'prob': all_probs, 'label': all_targets})
df_res = df_res.sort_values('prob', ascending=False)

# --- [1] Summary Metrics ---
roc_auc = roc_auc_score(all_targets, all_probs)
ap_score = average_precision_score(all_targets, all_probs)
brier = brier_score_loss(all_targets, all_probs)

# KS Statistic
class0_probs = df_res[df_res['label'] == 0]['prob']
class1_probs = df_res[df_res['label'] == 1]['prob']
ks_stat, ks_pval = ks_2samp(class1_probs, class0_probs)

summary_df = pd.DataFrame({
    'Metric': ['ROC-AUC', 'Average Precision (AP)', 'KS Statistic', 'Brier Score'],
    'Value': [roc_auc, ap_score, ks_stat, brier],
    'Interpretation': [
        '전체적인 변별력 (0.5=랜덤, 1.0=완벽)',
        '소수 클래스(이탈자) 찾기 실력',
        '두 집단(이탈/유지)의 분리 정도 (최대 차이)',
        '확률값의 정확도 (0에 가까울수록 좋음)'
    ]
})
print('\n[Summary Metrics]')
print(summary_df.round(4).to_string(index=False))

# --- [2] Ranking Performance (Top K%) ---
base_rate = df_res['label'].mean()
results = []
thresholds_pct = [5, 10, 15, 30]
for k in thresholds_pct:
    top_k_count = int(len(df_res) * (k/100))
    top_k_df = df_res.iloc[:top_k_count]
    precision = top_k_df['label'].mean()
    lift = precision / base_rate
    results.append({'Top K%': f'{k}%', 'Precision': f'{precision*100:.2f}%', 'Lift': f'{lift:.2f}x'})

print('\n[Ranking Performance]')
print(pd.DataFrame(results).to_string(index=False))

# --- [3] Dashboard Visualization (2x2) ---
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
plt.subplots_adjust(hspace=0.3)

# (1) ROC Curve
fpr, tpr, _ = roc_curve(all_targets, all_probs)
axes[0, 0].plot(fpr, tpr, label=f'AUC = {roc_auc:.4f}', color='darkorange', lw=2)
axes[0, 0].plot([0, 1], [0, 1], 'k--', alpha=0.5)
axes[0, 0].set_title('ROC Curve (General Discrimination)')
axes[0, 0].set_xlabel('False Positive Rate')
axes[0, 0].set_ylabel('True Positive Rate')
axes[0, 0].legend(loc='lower right')
axes[0, 0].grid(True, alpha=0.3)

# (2) PR Curve
precision_curve, recall_curve, _ = precision_recall_curve(all_targets, all_probs)
axes[0, 1].plot(recall_curve, precision_curve, label=f'AP = {ap_score:.4f}', color='green', lw=2)
axes[0, 1].axhline(y=base_rate, color='r', linestyle='--', label=f'Base Rate ({base_rate*100:.1f}%)')
axes[0, 1].set_title('PR Curve (Focus on Churners)')
axes[0, 1].set_xlabel('Recall (Detection Rate)')
axes[0, 1].set_ylabel('Precision (Accuracy)')
axes[0, 1].legend(loc='upper right')
axes[0, 1].grid(True, alpha=0.3)

# (3) Calibration Curve
prob_true, prob_pred = calibration_curve(all_targets, all_probs, n_bins=10)
axes[1, 0].plot(prob_pred, prob_true, marker='o', label='Model', color='blue')
axes[1, 0].plot([0, 1], [0, 1], 'k--', alpha=0.5, label='Perfectly Calibrated')
axes[1, 0].set_title('Calibration Curve (Reliability)')
axes[1, 0].set_xlabel('Mean Predicted Probability')
axes[1, 0].set_ylabel('Fraction of Positives')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# (4) Efficiency Curve (Top K%)
k_values = np.linspace(1, 100, 100)
precisions = []
for k in k_values:
    count = int(len(df_res) * (k/100))
    if count == 0: count = 1
    prec = df_res.iloc[:count]['label'].mean()
    precisions.append(prec)

axes[1, 1].plot(k_values, precisions, label='Model Precision', color='purple', lw=2)
axes[1, 1].axhline(y=base_rate, color='r', linestyle='--', label=f'Base Rate')
axes[1, 1].set_title('Targeting Efficiency (Business Value)')
axes[1, 1].set_xlabel('Top K% (Target Group Size)')
axes[1, 1].set_ylabel('Precision')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

# Annotation for Top K
for k in [5, 10, 30]:
    p = precisions[int(k)-1]
    axes[1, 1].plot(k, p, 'ro')
    axes[1, 1].text(k, p+0.05, f'{p*100:.1f}%', ha='center', fontweight='bold')

plt.tight_layout()
plt.show()

>>> Generating Advanced Verification Dashboard...

[Summary Metrics]
                Metric  Value             Interpretation
               ROC-AUC 0.6886  전체적인 변별력 (0.5=랜덤, 1.0=완벽)
Average Precision (AP) 0.8955          소수 클래스(이탈자) 찾기 실력
          KS Statistic 0.2714 두 집단(이탈/유지)의 분리 정도 (최대 차이)
           Brier Score 0.2108     확률값의 정확도 (0에 가까울수록 좋음)

[Ranking Performance]
Top K% Precision  Lift
    5%    93.36% 1.14x
   10%    93.11% 1.14x
   15%    92.39% 1.13x
   30%    91.23% 1.11x


In [ ]:
import os
import json
import torch
import pandas as pd
import matplotlib.pyplot as plt

print(">>> [System] Starting Artifact Saving Pipeline...")

# 1. Define Paths
PATH_MODELS = "../../models/dl/"
PATH_METRICS = "../../models/metrics/"
PATH_ASSETS = "../../assets/training/"
PATH_REPORTS = "../../reports/training/"

for path in [PATH_MODELS, PATH_METRICS, PATH_ASSETS, PATH_REPORTS]:
    os.makedirs(path, exist_ok=True)

# 2. Save Model
model_path = os.path.join(PATH_MODELS, "MLP_base_model.pt")
torch.save(model.state_dict(), model_path)

# 3. Save Metrics
metrics_data = {
    "summary": summary_df.to_dict(orient="records"),
    "ranking": results
}
with open(os.path.join(PATH_METRICS, "MLP_base_metrics.json"), "w", encoding="utf-8") as f:
    json.dump(metrics_data, f, ensure_ascii=False, indent=4)

# 4. Save Plots
fig.savefig(os.path.join(PATH_ASSETS, "MLP_base_dashboard.png"))

# 5. Save Report
report_content = f"# Final Benchmark Report - MLP_base\n\n## Summary Metrics\n{summary_df.to_markdown(index=False)}\n\n## Ranking Performance\n{pd.DataFrame(results).to_markdown(index=False)}"
with open(os.path.join(PATH_REPORTS, "MLP_base_report.md"), "w", encoding="utf-8") as f:
    f.write(report_content)

print("✅ [System] 모든 결과물이 ../../models/ 및 ../../reports/ 경로에 성공적으로 저장되었습니다.")